import glob

for fn in 

In [ ]:
import glob
import pickle
import numpy as np
from spf.utils import zarr_open_from_lmdb_store

fns = glob.glob("/mnt/4tb_ssd/precompute_cache_3p31/*.yarr")


def get_yarr_and_pkl(yarr_fn):
    yarr = zarr_open_from_lmdb_store(yarr_fn, mode="r")
    pkl = pickle.load(open(yarr_fn.replace(".yarr", ".pkl"), "rb"))
    return yarr, pkl


def nanstd(x):
    return x[np.isfinite(x)].std()


for fn in fns:
    try:
        yarr_3p3, pkl_3p3 = get_yarr_and_pkl(fn)
        yarr_3p11, pkl_3p11 = get_yarr_and_pkl(fn.replace("_3p31", ""))
        std0 = nanstd(
            yarr_3p3["r0"]["mean_phase"][:] - yarr_3p11["r0"]["mean_phase"][:]
        )
        std1 = nanstd(
            yarr_3p3["r1"]["mean_phase"][:] - yarr_3p11["r1"]["mean_phase"][:]
        )
        print(fn, std0, std1)
    except:
        pass

In [ ]:
import pickle
from spf.utils import zarr_open_from_lmdb_store

# yarr_fn = "/mnt/4tb_ssd/precompute_cache_3p31/precompute_cache_3p3/wallarrayv3_2024_10_10_22_53_15_nRX2_bounce_spacing0p025_segmentation_nthetas65.yarr/"
# yarr_fn = "/mnt/4tb_ssd/precompute_cache_3p31/wallarrayv3_2024_06_30_03_37_08_nRX2_rx_circle_spacing0p06_segmentation_nthetas65.yarr"
yarr_fn = "/mnt/4tb_ssd/precompute_cache_3p31/wallarrayv3_2024_06_06_15_16_06_nRX2_bounce_segmentation_nthetas65.yarr"
yarr_fn_3p11 = yarr_fn.replace("_3p4", "")
pkl_fn = yarr_fn.replace(".yarr", ".pkl")
pkl_3p11_fn = yarr_fn_3p11.replace(".yarr", ".pkl")
# yarr_fn = "/mnt/4tb_ssd/precompute_cache_3p3/wallarrayv3_2024_10_12_20_48_54_nRX2_rx_circle_spacing0p025_segmentation_nthetas65.yarr"
precomputed_zarr = zarr_open_from_lmdb_store(yarr_fn, mode="r", map_size=2**32)
precomputed_zarr_3p11 = zarr_open_from_lmdb_store(
    yarr_fn_3p11, mode="r", map_size=2**32
)
precomputed_pkl = pickle.load(open(pkl_fn, "rb"))
precomputed_3p11_pkl = pickle.load(open(pkl_3p11_fn, "rb"))

In [ ]:
precomputed_zarr["version"][:] = 3.31

In [ ]:
for idx in range(len(precomputed_pkl["segmentation_by_receiver"]["r0"])):
    a = precomputed_pkl["segmentation_by_receiver"]["r0"][idx]["simple_segmentation"]
    b = precomputed_3p11_pkl["segmentation_by_receiver"]["r0"][idx][
        "simple_segmentation"
    ]
    if len(a) != len(b):
        print(idx)
        break

In [ ]:
for x in b:
    print(x)

In [ ]:
for x in a:
    print(x)

In [ ]:
len(precomputed_pkl["segmentation_by_receiver"]["r0"][10]["simple_segmentation"][0])

In [ ]:
len(
    precomputed_3p11_pkl["segmentation_by_receiver"]["r0"][10]["simple_segmentation"][0]
)

In [ ]:
import torch
import numpy as np

from spf.rf import mean_phase_mean, torch_circular_mean, torch_mean_phase_mean

r_idx = 0
mean_phase = np.hstack(
    [
        (
            torch.tensor([x["mean"] for x in result["simple_segmentation"]]).mean()
            if len(result) > 0
            else torch.tensor(float("nan"))
        )
        for result in precomputed_pkl["segmentation_by_receiver"][f"r{r_idx}"]
    ]
)
# TODO THIS SHOULD BE FIXED!!!
mean_phase[~np.isfinite(mean_phase)] = 0


norm_mean_phase = np.hstack(
    [
        (
            torch_mean_phase_mean(
                torch.tensor([x["mean"] for x in result["simple_segmentation"]]),
                torch.ones(len(result["simple_segmentation"])),
            )
            if len(result) > 0
            else torch.tensor(float("nan"))
        )
        for result in precomputed_pkl["segmentation_by_receiver"][f"r{r_idx}"]
    ]
)
# TODO THIS SHOULD BE FIXED!!!
# norm_mean_phase[~np.isfinite(norm_mean_phase)] = 0

In [ ]:
mask = np.isfinite(norm_mean_phase)
mask.mean()

In [ ]:
from matplotlib import pyplot as plt

from spf.rf import pi_norm, torch_pi_norm

# bins=np.linspace(-8,8,50)
bins = np.linspace(-torch.pi, torch.pi, 50)
# plt.hist(
#     pi_norm(mean_phase - norm_mean_phase),
#     label="mean_phase-norm_mean_phase",
#     bins=bins,
#     histtype="step",
# )
# plt.hist(
#     pi_norm(mean_phase - precomputed_zarr["r0"]["mean_phase"][:]),
#     label="mean_phase-zarr",
#     bins=bins,
#     histtype="step",
# )
plt.hist(
    pi_norm(mean_phase - precomputed_zarr_3p11["r0"]["mean_phase"][:]),
    label="mean_phase-zarr_3p11",
    bins=bins,
    histtype="step",
)
# plt.hist(
#     pi_norm(norm_mean_phase - precomputed_zarr["r0"]["mean_phase"][:]),
#     label="norm_mean_phase-zarr",
#     bins=bins,
#     histtype="step",
# )
# plt.hist(
#     pi_norm(
#         precomputed_zarr_3p11["r0"]["mean_phase"][:]
#         - precomputed_zarr["r0"]["mean_phase"][:]
#     ),
#     label="3p11-3p3",
#     bins=bins,
#     histtype="step",
# )
plt.legend()

In [ ]:
precomputed_zarr["r0"]["mean_phase"][:]

In [ ]:
precomputed_zarr.tree()